In [ ]:
from model import SearchCifarNet
import torch

device = torch.device('cuda')

model = SearchCifarNet(n_classes=100, width_mult=1.0).to(device)
model.init_model()

In [ ]:
from putils import LabelSmoothingLoss, accuracy, get_parameters

momentum, nesterov = 0.9, True
optimizer = torch.optim.SGD(get_parameters(model), lr=0.05, momentum=momentum, nesterov=nesterov, weight_decay=4e-5)
loss_module = torch.nn.CrossEntropyLoss()

In [ ]:
from nni.retiarii.oneshot.pytorch import ProxylessTrainer
from torchvision import transforms
from torchvision.datasets import CIFAR100
dataset = CIFAR100(root='./cifar100', train=True, download=True, transform=transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
]))

train_loader = torch.utils.data.DataLoader(dataset, batch_size=256, num_workers=16)

In [4]:
from nni.retiarii.oneshot.pytorch.utils import AverageMeterGroup
from tqdm.auto import tqdm
model.train()

meters = AverageMeterGroup()
metrics_fn = lambda output, target: accuracy(output, target, topk=(1, 5,))

num_epochs = 100
for epoch in range(num_epochs):
    tr = tqdm(enumerate(train_loader))
    for step, (X_train, y_train) in tr:
        X_train = X_train.to(device)
        y_train = y_train.to(device)
        optimizer.zero_grad()
        logits = model(X_train)
        loss = loss_module(logits, y_train)
        loss.backward()
        optimizer.step()
        
        metrics = metrics_fn(logits, y_train)
        metrics['loss'] = loss.item()
        meters.update(metrics)
        
        if step % 50 == 0:
            tr.set_description("Epoch [%s/%s] Step [%s/%s]  %s" % (epoch + 1,
                                    num_epochs, step + 1, len(train_loader), meters))